In [163]:
# Import the required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from finta import TA
from pandas.tseries.offsets import DateOffset
import os
import requests
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, LSTM

%matplotlib inline

In [164]:
df = pd.read_csv("../algotrader2/resources/aapl_15min_pivot_point_indicator_df.csv")
df.head()

,timestamp,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,...,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal,new_signal
0,2023-01-09 21:30:00+00:00,130.0817,130.15,130.0400,314,130.09,18713,130.096350,0.000167,130.19634,...,-53.399594,-2.609553,False,-6.975113e+07,-22.378567,9.776626,1.066249,0.0,0,1
1,2023-01-09 21:45:00+00:00,130.1000,130.14,130.1000,182,130.11,4711,130.121440,0.000141,130.11634,...,-51.671029,-2.459362,False,-6.975348e+07,-23.502623,10.152024,1.046721,0.0,0,1
2,2023-01-09 22:00:00+00:00,130.0600,130.12,130.0300,128,130.12,6111,130.076383,-0.000307,130.09034,...,-52.958418,-2.603288,False,-6.976163e+07,-24.565646,9.791596,1.011924,0.0,0,1
3,2023-01-09 22:15:00+00:00,130.0700,130.09,130.0315,210,130.06,95004,130.135637,0.000077,130.07434,...,-51.834922,-2.888556,False,-6.972915e+07,-25.441689,10.514562,0.966759,0.0,0,1
4,2023-01-09 22:30:00+00:00,130.0600,130.08,130.0300,148,130.04,4724,130.045182,-0.000077,130.07434,...,-52.224826,-3.240848,False,-6.972537e+07,-26.410294,11.958663,0.907764,0.0,0,1


In [165]:
# Create our timestamp column as a datetime index, then save it as our index
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

# X is everything except the new_signal column
X = df.drop('new_signal', axis=1)

# We should use the .shift() function so that our algorithm predicts the minute before realtime
# Drop the row with NaN values 
X = X.shift().dropna()

# y is the NEW signal column
y = df[("new_signal")]

# Set start of training period
training_begin = X.index.min()

# we will train on 9 months and then test with the rest
training_end = X.index.min() + DateOffset(months=9)



In [166]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

In [167]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

display(X_train_scaled.shape)
display(X_test_scaled.shape)

(11440, 98)

(4046, 98)

In [168]:
num_predictors = len(X.columns)

# We have 2 possible outcomes, and we are trying to predict the stock/indicators to be in position -1 or 1
num_classes = 1

num_predictors

98

In [169]:
nn_model = Sequential()

In [170]:
# Add dense layer(s)
nn_model.add(Dense(units=16, input_dim=num_predictors, activation='relu'))
nn_model.add(Dense(units=32, activation='relu'))
nn_model.add(Dense(units=64, activation='relu'))

In [171]:
# Drop-out layer(s)
# nn_model.add(Dropout(.2,input_shape=(10,)))

# Add dense layer, add Regularization
#model.add(Dense(5, activation='relu', kernel_regularized=l2(0.01), bias_regularized=l2(0.01)))

In [172]:
# Add output layer
# Number of outputs equals number of classes
#nn_model.add(Dense(num_classes))
nn_model.add(Dense(num_classes, activation="sigmoid"))

In [173]:
# Compile model
nn_model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# Summarize model
nn_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 16)                1584      
                                                                 
 dense_28 (Dense)            (None, 32)                544       
                                                                 
 dense_29 (Dense)            (None, 64)                2112      
                                                                 
 dense_30 (Dense)            (None, 1)                 65        
                                                                 
Total params: 4305 (16.82 KB)
Trainable params: 4305 (16.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [174]:
# Fit model
num_epochs = 100

nn_model.fit(X_train_scaled, y_train,
          epochs=num_epochs,
          batch_size=32,
          validation_split=0.2,     # This 'validation_split' is telling the neural network to keep 20% of the data to validate its score on the training set... this is to help AVOID OVERFITTING. 
          shuffle=True)

Epoch 1/100


286/286 [==============================] - 0s 899us/step - loss: -13.6107 - accuracy: 0.0096 - val_loss: -78.2549 - val_accuracy: 0.0372
Epoch 2/100
286/286 [==============================] - 0s 603us/step - loss: -794.4243 - accuracy: 0.0799 - val_loss: -2299.3806 - val_accuracy: 0.0603
Epoch 3/100
286/286 [==============================] - 0s 631us/step - loss: -7038.9224 - accuracy: 0.0865 - val_loss: -13472.2051 - val_accuracy: 0.1040
Epoch 4/100
286/286 [==============================] - 0s 631us/step - loss: -29496.8594 - accuracy: 0.0927 - val_loss: -47731.2422 - val_accuracy: 0.1088
Epoch 5/100
286/286 [==============================] - 0s 594us/step - loss: -83348.7891 - accuracy: 0.1039 - val_loss: -116479.4609 - val_accuracy: 0.1119
Epoch 6/100
286/286 [==============================] - 0s 592us/step - loss: -181704.0781 - accuracy: 0.1003 - val_loss: -233927.9688 - val_accuracy: 0.1364
Epoch 7/100
286/286 [==============================] - 0s 600us/step - loss: -336624.8125

In [175]:
# Show model loss and accuracy

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

127/127 - 0s - loss: -2.8338e+09 - accuracy: 0.0976 - 102ms/epoch - 800us/step
Loss: -2833773824.0, Accuracy: 0.09762728959321976


In [176]:
# back test
# Predict values using testing data
nn_test_predictions = nn_model.predict(X_test_scaled)
nn_train_predictions = nn_model.predict(X_train_scaled)

358/358 [==============================] - 0s 271us/step


In [177]:
# EXTRA



# # Evaluate the model using a classification report
# from sklearn.metrics import classification_report
# training_report = classification_report(nn_train_predictions, nn_test_predictions)
# print(training_report)

# # Evaluate the model's ability to predict the trading signal for the testing data using a classification report
# training_report = classification_report(y_test, testing_signal_predictions)
# print(training_report)

# from imblearn.over_sampling import RandomOverSampler
# # Use RandomOverSampler to resample the datase using random_state=1
# ros = RandomOverSampler(random_state=1)
# X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

# Drop-out layer(s)
# nn_model.add(Dropout(.2,input_shape=(10,)))

# Add dense layer, add Regularization
#model.add(Dense(5, activation='relu', kernel_regularized=l2(0.01), bias_regularized=l2(0.01)))

# # Save model history for further manipulation
# model_history = model.history.model_history.keys()

# # Now we can plot the accuracy for training and validation

# training_results = pd.DataFrame(index=range(1, num_epochs+1))
# training_results['Training'] = model_history['categorical_accuracy']
# training_results['Validation'] = model_history['val_categorical_accuracy']
# training_results.plot(title = 'Training and Validation Performance')

In [178]:
display(X_train_scaled.shape)
display(X_test_scaled.shape)

(11440, 98)

(4046, 98)

In [179]:
num_samples = X_train_scaled.shape[0]
num_features = X_train_scaled.shape[1]

display(num_samples)
display(num_features)

11440

98

In [180]:
num_timesteps = 1  # This is an example; you might need a different value

In [181]:
# Reshape X_train_scaled and X_test_scaled
X_train_reshaped = X_train_scaled.reshape((num_samples, num_timesteps, num_features))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], num_timesteps, num_features))

In [182]:
display(X_train_reshaped)
display(X_test_reshaped)

array([[[-2.41770309e+00, -2.42670149e+00, -2.40594906e+00, ...,
          1.81155121e+00, -1.11872189e+00, -1.35846314e-03]],

       [[-2.41658341e+00, -2.42731372e+00, -2.40228144e+00, ...,
          1.75288220e+00, -1.11872189e+00, -1.35846314e-03]],

       [[-2.41903080e+00, -2.42853820e+00, -2.40656033e+00, ...,
          1.64834414e+00, -1.11872189e+00, -1.35846314e-03]],

       ...,

       [[ 5.67395127e-01,  5.63464862e-01,  5.75220682e-01, ...,
          4.57471385e-01,  1.12130670e+00, -2.22147536e+00]],

       [[ 5.69230669e-01,  5.57954728e-01,  5.78888310e-01, ...,
          1.88311378e-01,  1.09079146e+00, -1.35846314e-03]],

       [[ 5.66171433e-01,  5.56730254e-01,  5.75831953e-01, ...,
          5.26666840e-02,  1.00714243e+00, -1.35846314e-03]]])

array([[[ 5.66171433e-01,  5.56730254e-01,  5.75831953e-01, ...,
          5.26666840e-02,  1.00714243e+00, -1.35846314e-03]],

       [[ 5.64335891e-01,  5.54281305e-01,  5.75220682e-01, ...,
         -1.48337922e-01,  7.88218309e-01, -1.35846314e-03]],

       [[ 5.63112197e-01,  5.56118016e-01,  5.74609411e-01, ...,
         -3.97701095e-01,  3.56310828e-01, -1.35846314e-03]],

       ...,

       [[ 8.32324942e-01,  8.29175795e-01,  8.38678579e-01, ...,
         -2.55252057e-01,  1.12156143e+00,  2.21875844e+00]],

       [[ 8.29877553e-01,  8.24890135e-01,  8.39289850e-01, ...,
         -5.50127696e-01,  1.12156143e+00, -1.35846314e-03]],

       [[ 8.23147235e-01,  8.23665661e-01,  8.34399680e-01, ...,
         -7.15284152e-01,  1.11778458e+00, -1.35846314e-03]]])

In [183]:
X_train_reshaped.shape[1]

1

In [184]:
X_train_reshaped.shape[2]

98

### Trash model, let's change some stuff up.

In [185]:
num_predictors = len(X.columns)

# We have 2 possible outcomes, and we are trying to predict the stock/indicators to be in position -1 or 1
num_classes = 1

nn_model = Sequential()

# Add LSTM layer
nn_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
nn_model.add(Dropout(0.2))

# Add another LSTM layer
nn_model.add(LSTM(units=50, return_sequences=False))
nn_model.add(Dropout(0.2))

# Add dense layers
nn_model.add(Dense(units=64, activation='relu'))
nn_model.add(Dropout(0.2))

# Add output layer
# Number of outputs equals number of classes
#nn_model.add(Dense(num_classes))
nn_model.add(Dense(num_classes, activation="sigmoid"))

# Add output layer
nn_model.add(Dense(num_classes, activation="sigmoid"))

# Compile model
nn_model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# Summarize model
nn_model.summary()


# Fit model
num_epochs = 50
nn_model.fit(X_train_scaled, y_train, epochs=num_epochs, batch_size=32, validation_split=0.2, shuffle=True)


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 1, 50)             29800     
                                                                 
 dropout_6 (Dropout)         (None, 1, 50)             0         
                                                                 
 lstm_5 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_7 (Dropout)         (None, 50)                0         
                                                                 
 dense_31 (Dense)            (None, 64)                3264      
                                                                 
 dropout_8 (Dropout)         (None, 64)                0         
                                                                 
 dense_32 (Dense)            (None, 1)               

ValueError: in user code:

    File "/Users/montygash/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Users/montygash/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/montygash/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Users/montygash/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/Users/montygash/anaconda3/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/montygash/anaconda3/lib/python3.11/site-packages/keras/src/engine/input

### try removing the highly correlated things, use correlation matrix.. see Marghub's example project.

In [187]:
num_features = X_train_scaled.shape[1]

# Reshape the data to include the time steps dimension
# Here, we're using 1 time step as an example
X_train_reshaped = X_train_scaled.reshape(-1, 1, num_features)
X_test_reshaped = X_test_scaled.reshape(-1, 1, num_features)

# We have 2 possible outcomes, and we are trying to predict the stock/indicators to be in position -1 or 1
num_classes = 1

nn_model = Sequential()

# Add LSTM layer
nn_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
nn_model.add(Dropout(0.2))

# Add another LSTM layer
nn_model.add(LSTM(units=50, return_sequences=False))
nn_model.add(Dropout(0.2))

# Add dense layers
nn_model.add(Dense(units=64, activation='relu'))
nn_model.add(Dropout(0.2))

# Compile model
nn_model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# Summarize model
nn_model.summary()


# Fit model
num_epochs = 50
nn_model.fit(X_train_reshaped, y_train, epochs=num_epochs, batch_size=32, validation_split=0.2, shuffle=True)


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 1, 50)             29800     
                                                                 
 dropout_9 (Dropout)         (None, 1, 50)             0         
                                                                 
 lstm_7 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_10 (Dropout)        (None, 50)                0         
                                                                 
 dense_34 (Dense)            (None, 64)                3264      
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
Total params: 53264 (208.06 KB)
Trainable params: 532

Change time step from 1 to more.

This can be calculated by dividing the total number of elements in X_train_scaled by the number of features and then by the desired number of time steps.

In [193]:
# Calculate the total number of samples that can be formed with 100 time steps
num_samples = X_train_scaled.shape[0] // 1000

# Now reshape the array
X_train_reshaped = X_train_scaled[:num_samples * 1000].reshape(num_samples, 1000, num_features)

# Calculate the total number of samples that can be formed with 100 time steps
num_samples = X_test_scaled.shape[0] // 1000

# Now reshape the array
X_train_reshaped = X_test_scaled[:num_samples * 1000].reshape(num_samples, 1000, num_features)



In [194]:
num_features = X_train_scaled.shape[1]

# Reshape the data to include the time steps dimension
# -1 tells the program to keep the shape of our origional data set
# Let's change our time step from 1 to more; each step is 15 minutes given our data
#X_train_reshaped = X_train_scaled.reshape(-1, 100, num_features)
#X_test_reshaped = X_test_scaled.reshape(-1, 100, num_features)

# We have 2 possible outcomes, and we are trying to predict the stock/indicators to be in position -1 or 1
num_classes = 1

nn_model = Sequential()

# Add LSTM layer
nn_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
nn_model.add(Dropout(0.2))

# Add another LSTM layer
nn_model.add(LSTM(units=50, return_sequences=False))
nn_model.add(Dropout(0.2))

# Add dense layers
nn_model.add(Dense(units=64, activation='relu'))
nn_model.add(Dropout(0.2))

# Compile model
nn_model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# Summarize model
nn_model.summary()


# Fit model
num_epochs = 50
nn_model.fit(X_train_reshaped, y_train, epochs=num_epochs, batch_size=32, validation_split=0.2, shuffle=True)


Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 1000, 50)          29800     
                                                                 
 dropout_15 (Dropout)        (None, 1000, 50)          0         
                                                                 
 lstm_11 (LSTM)              (None, 50)                20200     
                                                                 
 dropout_16 (Dropout)        (None, 50)                0         
                                                                 
 dense_36 (Dense)            (None, 64)                3264      
                                                                 
 dropout_17 (Dropout)        (None, 64)                0         
                                                                 
Total params: 53264 (208.06 KB)
Trainable params: 532

### Well, at least the loss is positive... However, now our accuracy is non-existent! 